In [1]:
import numpy as np
import pandas as pd
import math

## Simulated Transactional Data

Next, we import the simulated transactional data, which records 1600 transactions over a 12-month period. These data have the following fields:

1. DATE: Date of transaction
2. COUNTRY BANK ORIGIN: Bank country originator transaction
3. RISK COUNTRY BANK ORIGIN: Risk of Bank country originator transaction	
4. COUNTRY BANK DESTINATION: Bank country reception transaction
5. RISK COUNTRY BANK DESTINATION: Risk of Bank country reception transaction
6. RISK COUNTRY: The highest risk of the countries involved in the transaction.	
7. OCCUPATION:	Occupation of customer involved in the transaction (Businessman,Employee,Housewife,Student)
8. OCCUPATION RISK: Risk of occupation 
9. CUSTOMER ID: Id of customer
10. AMOUNT: Transaction amount
11. CUSTOMER RISK:	Risk of customer (1-low,2-medium,3-high)
12. TRANSACTION TYPE: Type of transaction (Debit, Credit)
13. EYE: The variable that tells us whether a transaction was unusual or not (1 = Unusual, 0 = Usual).

In [2]:
df1 = pd.read_excel('txn_data.xlsx', sheet_name="Data")

In [3]:
df1.head()

,DATE,COUNTRY BANK ORIGIN,RISK COUNTRY BANK ORIGIN,COUNTRY BANK DESTINATION,RISK COUNTRY BANK DESTINATION,RISK COUNTRY,OCCUPATION,OCCUPATION RISK,CUSTOMER ID,AMOUNT2,CUSTOMER RISK,TRANSACTION TYPE,EYE
0,2022-09-24,Norway,Low,Cuba,High,3,Employee,1,2,56.780287,1,Debit,0
1,2022-06-08,Japan,Medium,Trinidad and Tobago,Medium,2,Employee,1,2,634.824687,1,Debit,0
2,2022-08-12,Japan,Medium,Cuba,High,3,Employee,1,2,40.930311,1,Credit,0
3,2022-12-07,Australia,Low,Thailand,Medium,2,Employee,1,2,425.857794,2,Debit,0
4,2022-08-22,Japan,Medium,Thailand,Medium,2,Employee,1,3,302.845597,1,Debit,0


## AHP Pairwise Matrix
Below is the pairwise matrix where the Saaty scale was used:

### Saaty's Comparison Scale
1. **1** EQUAL IMPORTANCE
2. **3** MODERATELY IMPORTANT
3. **5** STRONGLY IMPORTANT
4. **7** VERY STRONGLY IMPORTANT
5. **9** EXTREMELY IMPORTANT

We apply this scale to the factors:

1. BANK COUNTRY
2. OCCUPATION
3. CUSTOMER RISK

|   FACTORS     | BANK COUNTRY  | OCCUPATION | CUSTOMER RISK | 
|---------------|---------------|------------|---------------|
| BANK COUNTRY  | 1.000         | 3.000      | 7.000         | 
| OCCUPATION    | 0.333         | 1.000      | 3.000         |
| CUSTOMER RISK | 0.143         | 0.333      | 1.000         | 

This matrix recorded a consistency ratio of RC=0.05, therefore, we assume consistency and proceed with the calculation of weights that will allow us to convert the qualitative aspects of transactionality into a numerical variable.

## Weightings of the factors

|   FACTORS     |    WEIGHT     | 
|---------------|---------------|
| BANK COUNTRY  |     0.67      | 
| OCCUPATION    |     0.25      | 
| CUSTOMER RISK |     0.09      |

Under these weightings, the quantification of risk factors is calculated, and the **QPuntuation** field is obtained in the data matrix that we will use for unusual transaction detection. Additionally, the following variables are incorporated into this data matrix:

1. **TxJurisdiction**: An indicator that compares the transaction amount with the transactionality of the bank's country according to its risk.
2. **TxCustomer**: An indicator that compares the transaction amount with the transactionality of the customer.
3. **TxRisk**: An indicator that compares the transaction amount with the transactionality of customers according to their risk.
4. **TxOccupation**: An indicator that compares the transaction amount with the transactionality of customers according to their occupation.

In [4]:
df = pd.read_excel('txn_data.xlsx', sheet_name='PCAMatrix')

In [5]:
df.head()

,CUSTOMER ID,QPuntuation,TxJurisdiction,TxCustomer,TxRisk,TxOccupation,EYE
0,2,2.35,0.010981,0.195219,0.012913,0.009458,0
1,2,1.43,0.123338,2.182624,0.144375,0.105749,0
2,2,2.01,0.007916,0.140725,0.009309,0.006818,0
3,2,1.34,0.082739,1.464164,0.049360,0.070939,0
4,3,1.34,0.058839,0.775613,0.068875,0.050448,0


For the purposes of the project, we incorporate the variable "EYE" so that after applying PCA, we can validate the prediction of unusual transactions and, consequently, assess the efficiency of the method in detecting unusual aspects in them.

Next, we separate the quantitative variables that we will use for classification.

In [6]:
X, y  = df.iloc[:,1:6].values, df.loc[:,"EYE"].values

In [ ]:
mu = np.mean(X, axis=0)
sigma = np.std(X, axis=0)
X_std = (X - mu)/sigma

Desde aquí todo debe ser CAIRO

### <span style='color:Black'> **Generating Cairo files**  </span>

In [ ]:
def decimal_to_fp16x16(num):

    whole_num = int(num)
    fractional_part = int((num - whole_num) * 65536)
    fp_number = (whole_num << 16) + fractional_part
    return fp_number

In [ ]:
import os

In [ ]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
new_directory_path = os.path.join(parent_directory, "src/generated")

In [ ]:
os.makedirs('src/generated', exist_ok=True) 

In [ ]:
tensor_name = ["X","X_std","y"]  

def generate_cairo_files(data, name):

    with open(os.path.join('src', 'generated', f"{name}.cairo"), "w") as f:
        f.write(
            "use array::{ArrayTrait, SpanTrait};\n" +
            "use orion::operators::tensor::{core::{Tensor, TensorTrait}};\n" +
            "use orion::operators::tensor::FP16x16Tensor;\n" +
            "use orion::numbers::fixed_point::implementations::fp16x16::core::{FP16x16, FixedTrait};\n" +
            "\n" + f"fn {name}() -> Tensor<FP16x16>" + "{\n\n" + 
            "let mut shape = ArrayTrait::new();\n"
        )
        for dim in data.shape:
            f.write(f"shape.append({dim});\n")
    
        f.write("let mut data = ArrayTrait::new();\n")
        for val in np.nditer(data.flatten()):
            f.write(f"data.append(FixedTrait::new({abs(int(decimal_to_fp16x16(val)))}, {str(val < 0).lower()}));\n")
        f.write(
            "let tensor = TensorTrait::<FP16x16>::new(shape.span(), data.span());\n" +
            "return tensor;\n}"
        )

with open(f"src/generated.cairo", "w") as f:
    for n in tensor_name:
        f.write(f"mod {n};\n")

generate_cairo_files(X, "X")
generate_cairo_files(X_std, "X_std")
generate_cairo_files(y, "y")